## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [47]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [57]:
# 讀取鳶尾花資料集
digits = datasets.load_iris() #classifier data
#digits = datasets.load_boston()  #regression data

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=42)

# 建立模型
reg = GradientBoostingRegressor(random_state=7)


In [58]:
# 先看看使用預設參數得到的結果 for GradientBoostingRegressor (可用在Regression, Classifier data)
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
print("GB Regressor MSE : ", metrics.mean_squared_error(y_test, y_pred))


GB Regressor MSE :  0.004069046331210786


In [61]:
# 用 GradientBoostingClassifier 建立模型(無法用在Regression data)
reg_c = GradientBoostingClassifier(random_state=7)
reg_c.fit(x_train, y_train)
y_pred = reg_c.predict(x_test)
print("GB Classifier ACC : ", metrics.accuracy_score(y_test, y_pred))


GB Classifier ACC :  1.0


In [62]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型
# 印出最佳結果與最佳參數
print("GB Regression : Best Mean cross-validated score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

print("GB Regression : Best params:", grid_result.best_params_)


/Volumes/Data/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
GB Regression : Best Mean cross-validated score: -0.051120 using {'max_depth': 1, 'n_estimators': 50}
GB Regression : Best params: {'max_depth': 1, 'n_estimators': 50}


[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:    1.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.0s finished
/Volumes/Data/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [63]:
# 尋找超參數組合 for GradientBoostingClassifier
grid_search_c = GridSearchCV(reg, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)
grid_result_c = grid_search.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("GB Classifier : Best Mean cross-validated score: %f using %s" % (grid_result_c.best_score_, grid_result_c.best_params_))
print("GB Classifier : Best params: ", grid_result_c.best_params_)

/Volumes/Data/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
GB Classifier : Best Mean cross-validated score: -0.051120 using {'max_depth': 1, 'n_estimators': 50}
GB Classifier : Best params:  {'max_depth': 1, 'n_estimators': 50}


[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    0.6s finished
/Volumes/Data/Anaconda/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [64]:
# 使用最佳參數重新建立模型 for GradientBoostingRegressor
reg_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
reg_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = reg_bestparam.predict(x_test)

# 調整參數後看 MSE 是否降低
print("GB Regressor MSE : ", metrics.mean_squared_error(y_test, y_pred))


GB Regressor MSE :  0.008186294668339006


In [65]:
# 使用最佳參數重新建立模型 for GradientBoostingClassifier
reg_bestparam_c = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])
reg_bestparam_c.fit(x_train, y_train)
y_pred = reg_bestparam_c.predict(x_test)

#調整參數後的 ACC
print("GB Classifier ACC : ", metrics.accuracy_score(y_test, y_pred))


GB Classifier ACC :  1.0
